In [5]:
%matplotlib inline  
import matplotlib.pyplot as plt
from pamtra2.libs.singleScattering import scattering
from pamtra2.libs.singleScattering import mie, rayleigh, tmatrix, ssrg
from pamtra2.libs import refractiveIndex
import numpy as np
rad2deg = 180.0/np.pi

# Angles
t0 = np.pi*0.5 # theta must be within 0 pi
t1 = np.pi*0.5
#t1 = np.pi-t0
p0 = 0.0*np.pi
p1 = 0.8*np.pi
#p1 = np.pi+p0
print(p1, p1 // (2.*np.pi))
#p1 = p1 - 2.*np.pi*(p1//(2.*np.pi))
print(t0, t1, p0, p1)

2.5132741228718345 0.0
1.5707963267948966 1.5707963267948966 0.0 2.5132741228718345


In [6]:
# One centimeter particle, S-band 2.8GHz, pure ice sphere 
f = 2.8e9
d = 0.001
n = refractiveIndex.n(frequencies=f, temperatures=263.15, substance='ice')

cost=1.0
R = rayleigh.RayleighScatt(d,frequency=f,refractive_index=n, theta_inc=t0, theta_sca=t1, phi_inc=p0, phi_sca=p1)
M = mie.MieScatt(d*cost,frequency=f/cost,refractive_index=n, theta_inc=t0, theta_sca=t1, phi_inc=p0, phi_sca=p1)
T = tmatrix.TmatrixScatt(d*cost,frequency=f/cost,refractive_index=n, theta_inc=t0, theta_sca=t1, phi_inc=p0, phi_sca=p1)
S = ssrg.SsrgScatt(d,frequency=f,refractive_index=n, theta_inc=t0, theta_sca=t1, phi_inc=p0, phi_sca=p1, volume=np.pi*d**3/6.)

from pytmatrix import tmatrix as tm
from pytmatrix import scatter, radar
scatt = tm.Scatterer(radius=0.5*d, wavelength=299792458./f, m=n, axis_ratio=1.0/1.0, thet0=t0*rad2deg, thet=t1*rad2deg, phi0=p0*rad2deg, phi=p1*rad2deg)

print('Cext                    Csca                    Cabs                   Cbck            [m**2]')
print(R.Cext, R.Csca, R.Cabs, R.Cbck)
print(M.Cext, M.Csca, M.Cabs, M.Cbck)
print(S.Cext, S.Csca, S.Cabs, S.Cbck)
print(T.Cext, T.Csca, T.Cabs, T.Cbck)


print(" ")
print("RAY")
print(" ")
print(R.S2, complex(0.,0.))
print(complex(0.,0.), R.S1)
print(" ")
print(R.S2/(R.S1*np.cos(p1-p0)))
print(" ")
print("MIE")
print(M.S2,complex(0.,0.))
print(complex(0.,0.),M.S1)
print(" ")
print(M.S2/(M.S1*np.cos(p1-p0)))
print(" ")
print("SSRG")
print(" ")
print(S.S2, complex(0.,0.))
print(complex(0.,0.), S.S1)
print(" ")
print("TMM")
print(T.S[0,0], complex(0.,0.))
print(complex(0.,0.), T.S[1,1])
print(" ")
print(T.S[1,1]/(T.S[0,0]*np.cos(p1-p0)))
print(" ")
print(scatt.get_S())
print(" ")
print(T.S)
print(" ")
print(" ")


Cext                    Csca                    Cabs                   Cbck            [m**2]
3.42313882614e-12 2.74872213037e-13 3.1482666131e-12 4.12308319556e-13
3.42552492696e-12 2.74936863749e-13 3.15058806321e-12 4.12217233995e-13
3.42307506305e-12 2.748084499522858e-13 3.1482666131e-12 4.12117043789e-13
3.42552500089e-12 2.749368496528208e-13 3.15058815123e-12 4.12217233682e-13
 
RAY
 
(1.81136559953e-07+1.47020818594e-11j) 0j
0j (-1.46542555304e-07-1.18942340769e-11j)
 
(1.527864045+0j)
 
MIE
(1.8112443992210072e-07+1.5988074702946703e-11j) 0j
0j (-1.465247960607575e-07-1.293337276348207e-11j)
 
(1.5279469833729238+5.727790855102167e-09j)
 
SSRG
 
(1.81098551347e-07+1.4698996863e-11j) 0j
0j (-1.46511805696e-07-1.18917382624e-11j)
 
TMM
(1.81124439028e-07+1.59880749514e-11j) 0j
0j (-1.4652841631e-07-1.29336926371e-11j)
 
(0.99997043009-3.74817387135e-09j)
 
[[  1.81124439e-07 +1.59880750e-11j   6.51975072e-24 +5.75609846e-28j]
 [ -6.51975072e-24 -5.75609846e-28j  -1.46528416e-07

In [4]:
S.scattering_xsect()*0.5

2.748084499522858e-13

In [3]:
# One millimeter particle, W-band 94GHz, pure ice sphere 
f = 94e9
d = 0.001
n = refractiveIndex.n(frequencies=f, temperatures=263.15, substance='ice')
rayC = scattering(diameters=d, frequencies=f, refractive_indices=n, model='Ray')
mieC = scattering(diameters=d, frequencies=f, refractive_indices=n, model='Mie')
TmaC = scattering(diameters=d, frequencies=f, refractive_indices=n, model='TMM', theta_inc=t0, theta_sca=t1, phi_inc=p0, phi_sca=p1)

print('Cext                    Csca                    Cabs                   Cbck            [m**2]')
print(rayC)
print(mieC)
print(TmaC)
from pytmatrix import tmatrix, scatter, radar
scatt = tmatrix.Scatterer(radius=0.5*d, wavelength=299792458./f, m=n, axis_ratio=1.0/1.0, thet0=t0*rad2deg, thet=t1*rad2deg, phi0=p0*rad2deg, phi=p1*rad2deg)
print([scatter.ext_xsect(scatt),scatter.sca_xsect(scatt),scatter.ext_xsect(scatt)-scatter.sca_xsect(scatt),radar.radar_xsect(scatt)])

Cext                    Csca                    Cabs                   Cbck            [m**2]
(1.0498967461984795e-06, 1.0474543949973939e-06, 2.4423512010854983e-09, 5.2372719749869694e-07)
(3.8169026989341472e-07, 3.7719995847397647e-07, 4.4903114194382853e-09, 3.0198335883905455e-07)
(3.8169025547199783e-07, 3.771999482202861e-07, 4.4903072517117332e-09, 3.0198337240757998e-07)
[3.8169025547199783e-07, 3.771999482202861e-07, 4.4903072517117332e-09, 3.0198337240757998e-07]


In [4]:
# One millimeter particle, S-band 2.8GHz, pure ice sphere 
f=2.8e9
n = refractiveIndex.n(frequencies=f,temperatures=263.15,substance='ice')
rayC = scattering(diameters=0.0001, frequencies=f, refractive_indices=n, model='Ray')
mieC = scattering(diameters=0.0001, frequencies=f, refractive_indices=n, model='Mie')
TmaC = scattering(diameters=0.0001, frequencies=f, refractive_indices=n, model='TMM')

print('Cext                    Csca                    Cabs                   Cbck            [m**2]')
print(rayC)
print(mieC)
print(TmaC)

Cext                    Csca                    Cabs                   Cbck            [m**2]
(3.1490912297415749e-15, 8.2461663911100578e-19, 3.1482666131024637e-15, 4.1230831955550289e-19)
(3.1485646987497757e-15, 2.7487285968888451e-19, 3.1482898258900869e-15, 4.1230740899916799e-19)
(3.1485646291424952e-15, 2.7487285368539355e-19, 3.14828975628881e-15, 4.1230742079816574e-19)


In [5]:
scatt.get_S()
scatt.get_Z()

array([[  2.40310732e-08,   0.00000000e+00,   2.66798438e-24,
         -3.94430453e-31],
       [  0.00000000e+00,   2.40310732e-08,  -9.61242926e-15,
         -0.00000000e+00],
       [ -2.66798438e-24,  -9.61242926e-15,  -2.40310732e-08,
         -4.70197740e-38],
       [  3.94430453e-31,   0.00000000e+00,  -4.70197740e-38,
         -2.40310732e-08]])